# SQL Server Scripts

## Get schemas, tables and fields

In [ ]:
SELECT 
TABLE_SCHEMA
,TABLE_NAME
,COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE 1=1
--AND TABLE_SCHEMA = 'mdw'
--AND COLUMN_NAME NOT IN ('DW_Batch','DW_Id','DW_SourceCode','DW_TimeStamp','_Database','_DatabasePkey','_Id','_IncrementDatumTijd','_DatabaseFlorisoftIncrement','_DatabaseNummerExact','_DatabaseSysguid','_FeitType','_DatabaseFlorisoft')
--AND COLUMN_NAME NOT LIKE '%Key%'
ORDER BY
TABLE_SCHEMA
,TABLE_NAME
,COLUMN_NAME



## Get running processes

In [ ]:
SELECT 
r.session_id,
s.login_name,
c.client_net_address,
s.host_name,
s.program_name,
st.text,
s.status
FROM sys.dm_exec_requests r
INNER JOIN sys.dm_exec_sessions s ON r.session_id = s.session_id
LEFT JOIN sys.dm_exec_connections c ON r.session_id = c.session_id outer apply sys.dm_exec_sql_text(r.sql_handle) st
WHERE client_net_address is NOT null AND text is NOT NULL and s.status = 'running'
;

In [ ]:
-- https://dba.stackexchange.com/questions/231819/find-queries-executed-by-an-user-in-sql-server
SELECT 
sqltext.TEXT,
req.session_id,
req.status,
req.command,
req.cpu_time,
req.total_elapsed_time
FROM sys.dm_exec_requests req
CROSS APPLY sys.dm_exec_sql_text(sql_handle) AS sqltext

## Check and kill processes

In [ ]:
SP_who2 -- check processes
--kill 91;

## ?

In [ ]:
-- By default, it shows all processes in SQL Server.
SELECT * FROM sys.dm_exec_sessions;

In [ ]:
--We might not be interested in the system processes. We can filter the results using the following query.
SELECT *
FROM sys.dm_exec_sessions
WHERE is_user_process = 1 and status ='running';

## Get database sizes

In [ ]:
SELECT 
DatabaseName	= DB_NAME(database_id)
,LogSizeGB		= CAST(SUM(CASE WHEN type_desc = 'LOG' THEN size END) * 8. / 1024 / 1024 AS DECIMAL(8,2)) 
,RowSizeGB		= CAST(SUM(CASE WHEN type_desc = 'ROWS' THEN size END) * 8. / 1024 / 1024 AS DECIMAL(8,2)) 
,TotalSizeGB	= CAST(SUM(size) * 8. / 1024 / 1024 AS DECIMAL(8,2))
FROM sys.master_files WITH(NOWAIT)
WHERE 1 = 1 
--AND DB_NAME(database_id) LIKE 'TX%'
--AND database_id = DB_ID() -- for current db 
GROUP BY database_id
ORDER BY 1 ASC;

## Database file sizes

In [ ]:
SELECT 
cast(size as int) /1024/1024/1024
,*
FROM sys.master_files WITH(NOWAIT)
order by cast(size as int)  desc

## Table sizes

In [ ]:
SELECT 
TableName		    = t.NAME
,SchemaName		  = s.Name
,Rows			      = FORMAT(p.rows, '#,#')
,TotalSpaceGB	  = CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00 / 1024.00), 2) AS NUMERIC(36, 2))
,UsedSpaceGB	  = CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00 / 1024.00), 2) AS NUMERIC(36, 2))
,UnusedSpaceGB	= CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00 / 1024.00, 2) AS NUMERIC(36, 2))
FROM sys.tables t
INNER JOIN sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN sys.allocation_units a	ON p.partition_id	= a.container_id
LEFT OUTER JOIN sys.schemas s ON t.schema_id = s.schema_id
WHERE 1 = 1 
--AND 
GROUP BY
t.Name
,s.Name
,FORMAT(p.rows, '#,#')
ORDER BY TotalSpaceGB DESC ,t.Name
;

## Estimate data compression savings for table

In [ ]:
EXEC sp_estimate_data_compression_savings 'dsa', 'Factuurregel', NULL, NULL, 'ROW' ;  

## Rights (?)

In [ ]:
select @@ServerName [Server Name], DB_NAME() [DB Name], u.name [DB Role], u2.name [Member Name]
from sys.database_role_members m
join sys.database_principals u on m.role_principal_id = u.principal_id
join sys.database_principals u2 on m.member_principal_id = u2.principal_id
--where u.name = 'db_owner'
order by [Member Name]

## Check running queries on the database

In [ ]:
SELECT t.[text], s.last_execution_time
FROM sys.dm_exec_cached_plans AS p
INNER JOIN sys.dm_exec_query_stats AS s
   ON p.plan_handle = s.plan_handle
CROSS APPLY sys.dm_exec_sql_text(p.plan_handle) AS t
WHERE 1 = 1 
--AND t.[text] LIKE N'%SELECT @VolledigInladen%'
ORDER BY s.last_execution_time DESC;